In [1]:
!pip install openai
!pip install langchain-core
!pip install langchain-openai
!pip install langchain-community
!pip install chromadb
!pip install pypdf
!pip install pydantic==1.10.12


!pip install --upgrade openai
!pip install langchain==0.0.240
!pip install httpx==0.27.2


!pip install -q langchain
!pip install -q langchain-community
!pip install -q python-dotenv
!pip install -q huggingface
!pip install -q -U langchain-huggingface
!pip install -q transformers
!pip install -q sentence_transformers
!pip install -q tiktoken
!pip install -q -U langchain-chroma
!pip install -q streamlit
!pip install -q torch


import langchain, langchain_community, os, shutil, getpass, torch
from huggingface_hub import login
#from langchain.document_loaders import DirectoryLoader
from transformers import pipeline, AutoTokenizer, AutoModel
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.evaluation import load_evaluator
import streamlit as st

import openai
import os

  Using cached pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
Using cached pydantic-1.10.12-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydantic-2.10.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires pydantic>=2.7.0, but you have pydantic 1.10.12 which is incompatible.
gradio 5.8.0 requires pydantic>=2.0, but you have pydantic 1.10.12 which is incompatible.
langchain 0.3.10 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.12 which is incompatible.
langchain-core 0.3.22 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.12 which is incompatible.
pydantic-settings 2.6

In [2]:
f = open('/content/ts_openapi_key.txt')
api_key = f.read()
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI

from langchain_community.document_loaders import PyPDFLoader
# Load PDF
loaders = [
    # Duplicate documents on purpose
    PyPDFLoader("/content/context_kb.pdf"),

]
docs = []

for loader in loaders:
    docs.extend(loader.load())


#print(docs[0].page_content)

for loader in loaders:
    docs.extend(loader.load())

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split
#from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

splits = text_splitter.split_documents(docs)
print(len(splits))
print(len(splits[0].page_content) )
#splits[0].page_content

32
413


In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')


from langchain_community.vectorstores import Chroma # Light-weight and in memory

persist_directory = 'docs/chroma_6/'

<ipython-input-5-306f8d0a6ae8>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model='text-embedding-ada-002')


In [9]:
###
vectordb = Chroma.from_documents(
    documents=splits, # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory, # save the directory
)
###


#print(vectordb._collection.count()) # same as number of splites

retriever = vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 7, "fetch_k":15})

llm_name = "gpt-4o-mini"
print(llm_name)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

from langchain_core.prompts import PromptTemplate     #
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Build prompt
template = """Use the following pieces of context to explain the openstack log. Explain what that log means and how to sovle it

  Always say "thanks for asking!" at the end of the answer.
  {context}
  Question: {question}
  Helpful Answer:"""


def ragFunction(question):

  QA_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)
  rag_chain= {"context":RunnablePassthrough(context= lambda x:x["question"] | retriever),
         "question": lambda x:x["question"]}|QA_PROMPT | llm |StrOutputParser()

  print(question)
  #response=rag_chain.invoke(question)
  #response=rag_chain.invoke({"question" :"what is a package?"})
  response=rag_chain.invoke({"question" : question})
  return response


gpt-4o-mini


In [7]:
!pip install gradio

In [10]:
import gradio as gr
demo=gr.Interface(fn=ragFunction,inputs="text",outputs="text")
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41b13270fcbef82778.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
